# 1. Importation des packages

In [ ]:
# Importer les bibliothèques nécessaires pour l'analyse des données, la visualisation, le prétraitement, et la modélisation.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import warnings
warnings.filterwarnings('ignore')

# 2. Charger les données

## 2.1. Description de la source de données

In [ ]:
# Les données proviennent d'un fichier CSV (delhi_aqi.csv) contenant des mesures horaires de la qualité de l'air à Delhi.
# Les colonnes incluent des mesures de polluants tels que CO, NO, NO2, O3, SO2, PM2.5, PM10, NH3, ainsi que la date et l'heure.

## 2.2. Description des données

In [ ]:
# Charger les données dans un DataFrame Pandas.
# Afficher les premières lignes pour comprendre la structure.
# Vérifier les informations de base (types de données, valeurs manquantes, etc.).

In [ ]:
data = pd.read_csv('Data/delhi_aqi.csv')
print(data.head())
print(data.info())
print(data.isnull().sum())

# 3. Analyse des données (EDA)

In [ ]:
sns.histplot(data['so2'], kde=True)
plt.title('Distribution de SO2')
plt.show()

data['date'] = pd.to_datetime(data['date'])
plt.plot(data['date'], data['so2'])
plt.title('Tendance de SO2 au fil du temps')
plt.show()

sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.title('Matrice de corrélation')
plt.show()

# 4. Prétraitement des données

In [ ]:
# Extraction des caractéristiques temporelles

data['hour'] = data['date'].dt.hour
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month

# Normalisation des données
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data.drop(columns=['date']))

# 5. Division de données (Train et Test)

In [ ]:
# Normaliser les données
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data.drop(columns=['date']))

# Convertir en DataFrame avec les noms de colonnes
column_names = data.drop(columns=['date']).columns
scaled_data_df = pd.DataFrame(scaled_data, columns=column_names)

# Diviser les données en ensembles d'entraînement et de test
X = scaled_data_df.drop(columns=['so2'])  # Features
y = scaled_data_df['so2']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Construire le modele

## 6.1. Construire le modele Machine Learning

In [ ]:
# Nous allons utiliser un modèle de régression (Random Forest), Entraîner le modèle sur les données d'entraînement.
# Puis evaluer les performances sur les données de test.

model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print('R2 Score:', r2_score(y_test, y_pred_rf))

## 6.2. Construire le modele Deep Learning

In [ ]:
# Ici nous allons créer un modèle de réseau de neurones avec TensorFlow/Keras.
# Entraîner le modèle sur les données d'entraînement.
# Évaluer les performances sur les données de test

model_dl = Sequential()
model_dl.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model_dl.add(Dense(32, activation='relu'))
model_dl.add(Dense(1, activation='linear'))
model_dl.compile(optimizer='adam', loss='mse')
model_dl.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)
y_pred_dl = model_dl.predict(X_test)
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred_dl)))
print('R2 Score:', r2_score(y_test, y_pred_dl))

# 7. Visualiser l'evolution de l'entrainement du réseau de neurone

In [ ]:
# Visualiser les prédictions par rapport aux valeurs réelles

plt.scatter(y_test, y_pred_rf, label='Random Forest')
plt.scatter(y_test, y_pred_dl, label='Deep Learning')
plt.xlabel('Valeurs réelles')
plt.ylabel('Prédictions')
plt.legend()
plt.show()

# 8. Evaluer le modele

In [ ]:
# Comparons maintenant les performances des modèles (Machine Learning vs Deep Learning).
# Analyser les erreurs RMSE.

# Compiler le modèle avec une métrique supplémentaire
model_dl.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Entraîner le modèle et capturer l'historique
history = model_dl.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Visualiser la perte
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, 'bo-', label='Perte d\'entraînement')
plt.plot(epochs, val_loss, 'ro-', label='Perte de validation')
plt.title('Évolution de la perte pendant l\'entraînement')
plt.xlabel('Époques')
plt.ylabel('Perte')
plt.legend()
plt.show()

# Visualiser l'erreur absolue moyenne (MAE)
train_mae = history.history['mae']
val_mae = history.history['val_mae']

plt.plot(epochs, train_mae, 'bo-', label='MAE d\'entraînement')
plt.plot(epochs, val_mae, 'ro-', label='MAE de validation')
plt.title('Évolution de l\'erreur absolue moyenne (MAE)')
plt.xlabel('Époques')
plt.ylabel('MAE')
plt.legend()
plt.show()

